In [2]:
import re

only_digits = re.compile("[0-9]+")

def clear_only_digits(x):
    return ''.join(only_digits.findall(x))

print (clear_only_digits('81-sdf-2345-gdfg234-43h-h'))

81234523443


# parse_value

In [ ]:
re_number = re.compile("[0-9,.]+")
re_dotcomma = re.compile("[,.]+")

def parse_value(v):
    try:
        # Cliente digitou R4 no lugar de R$
        v = v.replace(u'R4 ',u'')
        
        v = re_number.findall(v)
        if not v:
            return None

        v = v[0]
        
        if (v.endswith(u',00') or v.endswith(u'.00') or v.endswith(u';00')):
            v = v[:-3]
    
        # re_dotcomma.split("1.121.990,00")
        # ['1', '121', '990', '00']
        valores = re_dotcomma.split(v)

        # re_dotcomma.findall("1.121.990,00")
        # ['.', '.', ',']
        sinais = re_dotcomma.findall(v)

        # print(valores)
        # print(sinais)

        # ex: 1234
        if len(valores) == 1:
            return float(valores[0])

        # Ex:
        # 12.3 -> decimal
        # 12.33 _ decimal
        # 1.000 -> ambiguo: Pode ser um ou mil. Assumindo mil
        # 1.0000 -> decimal
        # 123.000 -> ambiguo, assumindo 123 mil
        # 1234.000 -> 1234 mil
        if len(valores) == 2:
            l1 = len(valores[0])
            l2 = len(valores[1])
            if l1 >= 4:
                # assume que o simbolo é decimal porque temos mais de 4 numeros juntos
                return float(".".join(valores))

            if l2 == 3: 
                # assume milhar
                return float("".join(valores))

            return float(".".join(valores))

        # len(valores) > 2

        count_vigulas = 0
        count_pontos = 0
        for s in sinais:
            if s == ',':
                count_vigulas += 1
            else:
                count_pontos += 1

        if count_vigulas > 1 and count_pontos > 1:
            #nao sei... 
            return None

        sep_decimal = None
        sep_milhar = None

        if count_vigulas == 0:
            # Ex: 123.345.678
            sep_milhar = '.'

        elif count_pontos == 0:    
            # Ex: 123,345,678    
            sep_milhar = ','

        else:    
            # Ex: 123.123,12
            sep_decimal = sinais[-1]
            sep_milhar = sinais[-2]

        if sep_milhar:
            v = v.replace(sep_milhar, '')

        if sep_decimal:
            v = v.replace(sep_decimal, '.')
        
        return float(v)
    except:
        return None
